In [ ]:
# uncomment the below lines to run it on google colab
#from google.colab import drive
#drive.mount('/content/drive')
#import sys
#sys.path.append('/content/drive/MyDrive/research/Infinite/')

In [ ]:
! pip install transformers
! pip install seqeval
! pip install pytorch-lightning
! pip install pytorch_metric_learning

In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import seed_everything, loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import  DistilBertModel,DistilBertTokenizerFast

from pytorch_metric_learning import miners, losses

import random
import pandas as pd
from copy import deepcopy
from collections import OrderedDict

from scripts.dataset import *
from scripts.utils import *

In [ ]:
# basePath = '/content/drive/MyDrive/research/Infinite/' # uncomment to run it on colab
basePath = './'

In [ ]:
# model parameter
config = {
'mc' : {
    'model_name' : 'distilbert-base-cased',
    'tokenizer_name' : 'distilbert-base-cased',
    'joint_loss_coef' : 0.5,
    'id_1': 0.3986226896162205,
    'id_2':0.24620174329575167,
    'sd': 0.28096911585616136,
    'Ihs': 181
},

# training parameters
'tc' : {
    'lr' : 0.00002,
    'epoch' : 22,
    'batch_size' : 32,
    'weight_decay' : 0.003,
    'shuffle_data' : True,
    'num_worker' : 1
},

# data params

'dc' : {
    "train_dir": basePath + 'data/SNIPS/experiments/train/clean/train.tsv',
    "val_dir": basePath + 'data/SNIPS/experiments/dev/clean/dev.tsv',
    'intent_num' : 18,
    'slots_num' : 120,
    'max_len' : 56
},

# misc
'misc' : {
    'fix_seed' : False,
    'gpus' : 0,
    'param_dir' : basePath +  'bin/SNIPS/HCL/exp3/',
    'log_dir' : basePath + 'logs/SNIPS/HCL/exp3/',
    'precision' : 16
}
}

seed_everything(42)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased")

# loading background noise samples
with open(basePath + "data/BG_Noise_Phrase.txt") as f:
    content = f.readlines()
phrase = [x.strip() for x in content]

In [ ]:
# loading slot index file
tb_logger = pl_loggers.TensorBoardLogger(config['misc']['log_dir'])

checkpoint_callback = ModelCheckpoint(
    monitor='valid_IC_NER',
    dirpath= config['misc']['param_dir'],
    filename='INF_SCT1-{epoch:02d}-{val_loss:.2f}',
    save_top_k = 1,
)

final_slots = pd.read_csv( basePath + 'data/ATIS/slots_list.tsv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
idx2slots  = {idx:slots for idx,slots in enumerate(final_slots)}

In [1]:
# contrastive noise augmentation samples
def mergelistsBG(s, ns, prob):

    noisySample = deepcopy(ns)
    sample = deepcopy(s)

    bernaulliSample = [0] * int((1000) * prob) + [1] * int(1000 * (1 - prob))
    random.shuffle(bernaulliSample)

    final = []

    while len(noisySample) > 0 and len(sample) > 0:

        if random.sample(bernaulliSample, 1)[0] == 0:
            final.append(noisySample.pop(0))
        else:
            final.append(sample.pop(0))

    if len(noisySample) == 0:
        final = final + sample
    else:
        final = final + noisySample

    return s, final

def mergelistsMC(s, prob):

    anno = deepcopy(s)

    bernaulliSample = [0] * int((1000) * prob) + [1] * int(1000 * (1 - prob))
    random.shuffle(bernaulliSample)

    orig, aug = [anno[0]], [anno[0]]
    for tokens in anno[1:]:

        if random.sample(bernaulliSample, 1)[0] == 0:
            orig.append([tokens[0], 1000000])
        else:
            orig.append(tokens)
            aug.append(tokens)

    return orig, aug

def contrastiveSampleGeneration(sample):

    samplePacked = [[token, idx] for idx, token in enumerate(sample.split())]

    noisyTEXT = random.sample(phrase, 3)
    noisyTEXT = (noisyTEXT[0] + noisyTEXT[1] + noisyTEXT[2]).split(" ")
    noisyTOKENS = random.sample(noisyTEXT, random.sample([5, 6, 7, 8, 9, 10], 1)[0])
    noisyPacked = [[token, 2000000] for idx, token in enumerate(noisyTOKENS)]

    noiseType = random.sample(["MC", "BG"], 1)[0]
    noiseLevel = random.sample([0.25, 0.50, 0.75], 1)[0]

    if noiseType == "MC":
        orig, aug = mergelistsMC(samplePacked, prob=noiseLevel)
        augText, augSlots = zip(*aug)
        origText, origSlots = zip(*orig)

        return (
            " ".join(origText),
            " ".join(list(augText)),
            list(origSlots),
            list(augSlots),
        )
    else:
        orig, aug = mergelistsBG(samplePacked, noisyPacked, prob=noiseLevel)
        augText, augSlots = zip(*aug)
        origText, origSlots = zip(*orig)

        return (
            " ".join(list(origText)),
            " ".join(list(augText)),
            list(origSlots),
            list(augSlots),
        )

In [ ]:
def processSlotLabel(self, word_ids, slot_ids):

    # replace None and repetition with -100
    word_ids = [-100 if word_id == None else word_id for word_id in word_ids]
    previous_word = -100
    for idx, wid in enumerate(word_ids):

        if wid == -100:
            continue
        if wid == previous_word:
            word_ids[idx] = -100
        previous_word = wid

    Pslot_ids = []
    for sid in slot_ids.split():
        Pslot_ids.append(int(sid))

    new_labels = [-100 if word_id == -100 else Pslot_ids[word_id] for word_id in word_ids
    return new_labels

def batch_tokenizer(text, slotsID):
    token_ids, mask, slot_out = [], [], []
    for idx, sampleText in enumerate(text):
        inputs = tokenizer.encode_plus(
            sampleText.split(" "),
            None,
            add_special_tokens=True,
            return_token_type_ids=False,
            is_split_into_words=True,
            truncation=True,
            max_length=56,
            padding="max_length",
        )
        word_ids = inputs.word_ids()
        slot_out.append(processSlotLabel(word_ids, slotsID[idx]))

        token_ids.append(inputs["input_ids"])
        mask.append(inputs["attention_mask"])

    return token_ids, mask, slot_out

def list2Tensor(items):
    tensorItems = []
    for i in items:
        tensorItems.append(torch.tensor(i, dtype=torch.long))

    return tensorItems[0], tensorItems[1], tensorItems[2], tensorItems[3]

In [ ]:
def contrastivePairs(text):

    textP1, textP2, slotsID1, slotsID2, sentID1, sentID2 = [], [], [], [], [], []

    for idx, sample in enumerate(text):

        origText, augText, origSlots, augSlots = contrastiveSampleGeneration(sample)

        textP1.append(origText)
        slotsID1.append(origSlots)
        sentID1.append(idx)

        textP2.append(augText)
        slotsID2.append(augSlots)
        sentID2.append(idx)

    return textP1, textP2, slotsID1, slotsID2, sentID1, sentID2

In [ ]:
def train_collate(batch):

    data = {key: [] for key in batch[0].keys()}
    temp_ = {data[key].append(sample[key]) for sample in batch for key in sample.keys()}

    # processing batch for supervised learning
    # tokenization and packing to torch tensor
    token_ids, mask, slots_out = batch_tokenizer(data["TEXT"], data["slotsID"])
    token_ids, mask, intent_id, slots_out = list2Tensor(
        [token_ids, mask, data["intentID"], slots_out]
    )

    supBatch = {
        "token_ids": token_ids,
        "mask": mask,
        "intentId": intent_id,
        "slotsId": slots_out,
    }

    # processing batch for hierarchial contrastive learning

    # generating contrastive pairs
    textP1, textP2, tokenID1, tokenID2, sentID1, sentID2 = contrastivePairs(
        data["TEXT"]
    )

    # tokenization and packing for pair 1
    token_ids1, mask1, token_out1 = batch_tokenizer(textP1, tokenID1)
    token_ids1, mask1, sentID1, token_out1 = list2Tensor(
        [token_ids1, mask1, sentID1, token_out1]
    )

    # tokenization and packing for pair 2
    token_ids2, mask2, token_out2 = batch_tokenizer(textP2, tokenID2)
    token_ids2, mask2, sentID2, token_out2 = list2Tensor(
        [token_ids2, mask2, sentID2, token_out2]
    )

    CP1 = {
        "token_ids": token_ids1,
        "mask": mask1,
        "sentId": sentID1,
        "tokenId": token_out1,
    }

    CP2 = {
        "token_ids": token_ids2,
        "mask": mask2,
        "sentId": sentID2,
        "tokenId": token_out2,
    }

    return {"supBatch": supBatch, "HCLBatch": [CP1, CP2]}


def val_collate(batch):

    data = {key: [] for key in batch[0].keys()}
    _ = {data[key].append(sample[key]) for sample in batch for key in sample.keys()}

    # tokenization and packing to torch tensor
    token_ids, mask, slots_out = batch_tokenizer(data["TEXT"], data["slotsID"])
    token_ids, mask, intent_id, slots_out = list2Tensor(
        [token_ids, mask, data["intentID"], slots_out]
    )

    supBatch = {
        "token_ids": token_ids,
        "mask": mask,
        "intentId": intent_id,
        "slotsId": slots_out,
    }
    return {"supBatch": supBatch}

In [ ]:
class HCLDataset(Dataset):
    def __init__(self, file_dir):

        self.data = pd.read_csv(file_dir, sep="\t", header=0)

    def process_text(self, text):
        text = text.replace(".", "")
        text = text.replace("'", "")
        text = " ".join(text.split())
        return text

    def __getitem__(self, index):

        text = str(self.data.TEXT[index])
        text = self.process_text(text)

        sampleID = self.data.ID[index]

        intentID = self.data.INTENT_ID[index]

        slotsID = [int(sid) for sid in self.data.SLOTS_ID[index].split(" ")]

        return {
            "TEXT": text,
            "sampleID": sampleID,
            "intentID": intentID,
            "slotsID": slotsID,
        }

    def __len__(self):
        return len(self.data)


class HCLDataset_pl(pl.LightningDataModule):
    def __init__(self, train_dir, val_dir, batch_size, num_worker):

        super().__init__()
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.batch_size = batch_size
        self.num_worker = num_worker

    def setup(self, stage: [str] = None):
        self.train = HCLDataset(self.train_dir)

        self.val = HCLDataset(self.val_dir)

    def train_dataloader(self):
        return DataLoader(
            self.train, batch_size=self.batch_size, collate_fn=train_collate
        )

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=18, collate_fn=val_collate)


In [ ]:
class hierContrastiveICNER(nn.Module):
    def __init__(self, cfg):
        super(hierContrastiveICNER, self).__init__()

        self.encoder = DistilBertModel.from_pretrained(
            cfg['mc']['model_name'], return_dict=True, output_hidden_states=True
        )
        intentPred = OrderedDict([
            ('Relu1',nn.GELU()),
            ('dropout1',nn.Dropout(cfg["mc"]["id_1"])),
            ('iFC1',nn.Linear(768, cfg["mc"]["Ihs"])),
            ('Relu2',nn.GELU()),
            ('dropout2',nn.Dropout(cfg["mc"]["id_2"])),
            ('iFC2',nn.Linear(cfg["mc"]["Ihs"], cfg["dc"]["intent_num"]))
        ])
        self.intentPredMLP = nn.Sequential(intentPred)
        
        slotPred = OrderedDict([
            ('Relu1',nn.GELU()),
            ('dropout1',nn.Dropout(cfg["mc"]["sd"])),
            ('sFC1',nn.Linear(768, cfg["dc"]["slots_num"]))
        ])
        self.slotsPredMLP = nn.Sequential(slotPred)
        
        sentProjectionMLP = OrderedDict([
            ('FC1',nn.Linear(768, 768)),
            ('bn1',nn.BatchNorm1d(768)),
            ('Relu1',nn.GELU()),
            ('FC2',nn.Linear(768, 768)),
            ('bn2',nn.BatchNorm1d(768)),
            ('Relu2',nn.GELU())
        ])
        self.sentProjectMLP = nn.Sequential(sentProjectionMLP)
    
        tokenProjectionMLP = OrderedDict([
            ('FC1',nn.Linear(768, 768)),
            ('bn1',nn.BatchNorm1d(768)),
            ('Relu1',nn.GELU()),
            ('FC2',nn.Linear(768, 768)),
            ('bn2',nn.BatchNorm1d(768)),
            ('Relu2',nn.GELU())
        ])
        self.tokenProjectMLP = nn.Sequential(tokenProjectionMLP)
        
        self.criterion = nn.CosineSimilarity(dim=1)
        self.CE_criterion = nn.CrossEntropyLoss()

        self.jointICNERCoef = cfg["mc"]["joint_loss_coef"]
        self.jointHCLICNERCoef = 0.5
        self.cfg = cfg

    def CE_loss(self, encoded_output, intent_target, slots_target):

        # intent prediction loss
        intent_logits = self.intentPredMLP(encoded_output[0][:, 0])
        intent_loss = self.CE_criterion(intent_logits, intent_target)
        intent_pred = torch.argmax(nn.Softmax(dim=1)(intent_logits), axis=1)

        # slots prediction loss
        slots_logits = self.slotsPredMLP(encoded_output[0])
        slot_pred = torch.argmax(nn.Softmax(dim=2)(slots_logits), axis=2)
        
        slot_loss = self.CE_criterion(slots_logits.view(-1, self.cfg['mc']['slots_num']), 
                                      slots_target.view(-1))

        return {
            "ic_loss": intent_loss,
            "ner_loss": slot_loss,
            "intent_pred": intent_pred,
            "slot_pred": slot_pred,
        }

    def sentCL(self, sentz1, sentz2):

        # calculating sentence level loss
        p1, p2 = self.sentProjectMLP(sentz1), self.sentProjectMLP(sentz2)
        sentz2.detach()
        sentz1.detach()
        return  -(self.criterion(p2, sentz1).mean() + self.criterion(p1, sentz2).mean()) * 0.5
    
    def tokenCL(self, tokenEmb1,tokenEmb2,tokenID1,tokenID2):
        
        tokenID1 = torch.flatten(tokenID1)
        tokenID2 = torch.flatten(tokenID2)
        
        shape = tokenEmb1.shape
        tokenEmb1 = tokenEmb1.view(shape[0]*shape[1],-1)
        tokenEmb2 = tokenEmb2.view(shape[0]*shape[1],-1)

        filterTokenIdx1 = [idx for idx,val in enumerate(tokenID1.tolist()) if (val==-100 or val == 1000000 or val == 2000000)!=True]
        filterTokenIdx2 = [idx for idx,val in enumerate(tokenID2.tolist()) if (val==-100 or val == 1000000 or val == 2000000)!=True]
        
        filterTokenIdx1 = torch.tensor( filterTokenIdx1,dtype=torch.long,device=torch.device('cuda')) 
        filterTokenIdx2 = torch.tensor( filterTokenIdx2,dtype=torch.long,device=torch.device('cuda')) 
        
        tokenEmb1 = torch.index_select(tokenEmb1,0,filterTokenIdx1)
        tokenEmb2 = torch.index_select(tokenEmb2,0,filterTokenIdx2)
        
        # calculating sentence level loss
        p1, p2 = self.tokenProjectMLP(tokenEmb1), self.tokenProjectMLP(tokenEmb2)
        tokenEmb1.detach()
        tokenEmb2.detach()
        return  -(self.criterion(p2, tokenEmb1).mean() + self.criterion(p1, tokenEmb2).mean()) * 0.5

    def forward(self, batch , mode):

        if mode == "CE":
            encoded_output = self.encoder(batch['supBatch']['token_ids'], batch['supBatch']['mask'])
            return self.CE_loss(encoded_output, batch['supBatch']['intentId'], batch['supBatch']['slotsId'])

        if mode == "HCL":
            encoded_output_1 = self.encoder(batch['HCLBatch'][0]['token_ids'],batch['HCLBatch'][0]['mask']) 
            encoded_output_2 = self.encoder(batch['HCLBatch'][1]['token_ids'],batch['HCLBatch'][1]['mask']) 
            sentCL = self.sentCL(encoded_output_1[0][:, 0], encoded_output_2[0][:, 0])
            
            tokenIDs1 = batch['HCLBatch'][0]['tokenId']
            tokenIDs2 = batch['HCLBatch'][1]['tokenId']
            
            tokenCL = self.tokenCL(encoded_output_1[0], encoded_output_2[0],tokenIDs1,tokenIDs2)
            
            return {'jointCLLoss':(sentCL + tokenCL)*0.5}

In [ ]:
class TrainerHCL(pl.LightningModule):
    
    def __init__(self, cfg):
        super().__init__()
        
        self.model = hierContrastiveICNER(cfg)
        self.cfg = cfg
        self.steps = 0

    def forward(self, batch,mode):
        return self.model(batch,mode)
    
    def training_step(self, batch, batch_idx):

        self.steps += 1
        HCLLoss, flag = 0.0, 0
        # generating CCE loss
        try:
            HierConLoss = self(batch,'HCL')['jointCLLoss']
            self.log('jointCLLoss', HierConLoss, on_step=False, on_epoch=True, logger=True)
            if self.steps <= 500:
                return HierConLoss
        except:
            flag = 1

        # generating joint ICNER loss
        ICNER_out = self(batch,'CE')
        IC_loss, NER_loss = ICNER_out['ic_loss'],ICNER_out['ner_loss']
        
        self.log('IC_loss', IC_loss, on_step=False, on_epoch=True, logger=True)
        self.log('NER_loss', NER_loss, on_step=False, on_epoch=True, logger=True)
        
        if flag==0:
            return ((IC_loss + NER_loss)*0.50 + jointCLLoss)*0.50
        return
            return (IC_loss + NER_loss)*0.50
    
    def validation_step(self,batch,batch_idx):
        out = self(batch,'CE')
        IC_loss, NER_loss = out['ic_loss'], out['ner_loss']
        jointLoss = (IC_loss + NER_loss)*0.5
        self.log('valid_IC_NER', jointLoss, on_step=False, on_epoch=True, logger=True)
        return jointLoss
    
    def test_step(self,batch,batch_idx):
        out = self(batch,'CE')
        intent_pred, slots_pred = out['intent_pred'], out['slot_pred']
        intent_loss, slots_loss = out['ic_loss'], out['ner_loss']
        self.log('test_intent_acc', accuracy(intent_pred,batch['supBatch']['intentId']), on_step=False, on_epoch=True,  logger=True)
        self.log('test_slot_f1', slot_F1(slots_pred,batch['supBatch']['slotsId'],idx2slots), on_step=False, on_epoch=True, logger=True)
        return (intent_loss + slots_loss)*0.5
    
    def configure_optimizers(self):
         return torch.optim.AdamW( self.parameters(), lr=config['tc']['lr'] ,  weight_decay=self.cfg['tc']['weight_decay'])

In [ ]:
model = TrainerHCL(config)
contraDL = HCLDataset_pl(config['dc']['train_dir'], config['dc']['val_dir'],config['tc']['batch_size'],config['tc']['num_worker'])

In [ ]:
# model training
trainer = pl.Trainer(gpus=-1,deterministic=True, logger=tb_logger,callbacks=[checkpoint_callback] ,max_epochs=config['tc']['epoch'])

trainer.fit(model, contraDL)

In [ ]:
def slot_F1(pred,target,id2slots):

    pred_list = pred.tolist()
    target_list = target.tolist()
    
    pred_slots , target_slots = [],[]

    for idx_st,t in enumerate(target_list):
        pred_sample,target_sample = [],[]
        for idx_wt,wt in enumerate(t):

            if (wt == -100 or wt==0)!=True:
                assert(wt!=0)
                target_sample.append(id2slots[wt])
                pred_sample.append(id2slots[pred_list[idx_st][idx_wt]])

        pred_slots.append(pred_sample)
        target_slots.append(target_sample)
    
    return f1_score( target_slots, pred_slots,mode='strict', scheme=IOB2)

In [ ]:
 def cal_mean_stderror(metric):
    
    if len(metric) == 1:
        return metric
    var,std_error = 0,0
    mean = sum(metric)/len(metric)
    for m in metric:
        var += (m-mean)**2
    var = (var/(len(metric)-1))**0.5
    std_error = var/((len(metric))**0.5)
    return [round(mean,4),round(std_error,4)]

In [ ]:
test_50BG = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/50per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/50per/test_02.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/50per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/50per/test_04.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/50per/test_05.tsv']

test_25BG = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/25per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/25per/test_02.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/25per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/25per/test_04.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/25per/test_05.tsv']

test_75BG = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/75per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/75per/test_02.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/75per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/75per/test_04.tsv',
             '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseBG/75per/test_05.tsv']

test_20OOC = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/20per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/20per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/20per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/20per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/20per/test_05.tsv']

test_40OOC = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/40per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/40per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/40per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/40per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/40per/test_05.tsv']

test_60OOC = [ '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/60per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/60per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/60per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/60per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/NoiseOOC/60per/test_05.tsv']


test_clean = ['/content/drive/MyDrive/research/Infinite/data/SNIPS/experiments/test/clean/test.tsv']

In [ ]:
# testing the model

acc,slotF1 = [],[]

              
model_path = ['/content/drive/MyDrive/research/Infinite/bin/SNIPS/HCL/exp3/INF_SCT1-epoch=07-val_loss=0.00.ckpt',]


for m in model_path:
    model = TrainerHCL.load_from_checkpoint(m,cfg=config)
    
    for test_fn in test_clean:
    
        dm = HCLDataset_pl(test_fn,test_fn,16,1)
        dm.setup() 
        test = dm.val_dataloader()
        out = trainer.test(model=model ,test_dataloaders=test)
        acc.append(out[0]['test_intent_acc'])
        slotF1.append(out[0]['test_slot_f1'])
    
print('acc:',cal_mean_stderror(acc),'slotsF1',cal_mean_stderror(slotF1))